In [2]:
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 9.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.6/282.6 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 30.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.4 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 21.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/

In [9]:
import pandas as pd
import os
from transformers import BertTokenizer, BertModel
from torch import nn
import torch
import math
import textattack
import random
#from train_bert import Model

os.environ["CUDA_VISIBLE_DEVICES"] = ""
#torch.cuda.is_available = lambda : False
textattack.shared.utils.device = "cpu"

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        #self.bert_model.parallelize()
        self.drop = torch.nn.Dropout(p=0.1)
        self.l1 = torch.nn.Linear(768,2)

    def forward(self, text):
        tokenized_text = tokenizer(text , max_length=512, truncation=True, return_tensors='pt').input_ids#.to('cuda:3')
        text_rep = self.drop(self.bert_model(tokenized_text).pooler_output)
        out = self.l1(text_rep)
        #print(out)

        return out.squeeze().tolist()


model = Model()
#model.load_state_dict(torch.load('bert-base-uncased'))
model = model.to('cpu')
model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


class CustomWrapper(textattack.models.wrappers.ModelWrapper):
    def __init__(self, model):
        self.model = model#.to('cuda:3')
        self.model.eval()

    def __call__(self, list_of_texts):
        results = []
        self.model.requires_grad = False
        for text in list_of_texts:
          
          results.append(self.model(text))

        return results


class_model = CustomWrapper(model)


from textattack.datasets import Dataset
from textattack.attack_recipes.textfooler_jin_2019 import TextFoolerJin2019
from textattack import Attacker, AttackArgs


attack = TextFoolerJin2019.build(class_model)
attack#.cuda_()
        
dataset = [
    ["This film is a masterpiece! The story is incredibly moving, and the performances are outstanding. It's a true classic.", 1],
    ["The Godfather is a cinematic gem. The storytelling and performances are top-notch. A true classic in every sense.", 1],
    ["The Emoji Movie is a complete disappointment. The plot is weak, and it feels like one big advertisement. A waste of time.", 0],
    ["Mind-bending and visually stunning! Inception keeps you guessing from start to finish. Christopher Nolan at his best.", 1],
    ["Twilight is a guilty pleasure for some, but the acting and dialogue are cringe-worthy. Not a cinematic masterpiece.", 0],
    ["Forrest Gump is a heartwarming journey through history. Tom Hanks delivers an unforgettable performance.", 1],
    ["Explosions and CGI can't make up for the lackluster story in Transformers: The Last Knight. Disappointing.", 0],
    ["The Dark Knight is a dark and gripping superhero film. Heath Ledger's Joker is iconic. A must-see.", 1],
    ["Avatar is visually breathtaking, but the story is somewhat predictable. Still, it's a cinematic experience.", 1],
    ["The Room is so bad that it's almost good. The unintentional humor makes it a cult classic.", 1]
]

random.shuffle(dataset)

attacker = Attacker(attack, textattack.datasets.Dataset(dataset[:10]), AttackArgs(num_examples=10))
attacker.attack_dataset()

textattack: Unknown if model of class <class '__main__.Model'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


 10%|█         | 1/10 [00:48<07:13, 48.12s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  10%|█         | 1/10 [00:48<07:13, 48.12s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  20%|██        | 2/10 [00:48<03:12, 24.11s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:  20%|██        | 2/10 [00:48<03:12, 24.12s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (56%)]] --> [[[FAILED]]]

Explosions and CGI can't make up for the lackluster story in Transformers: The Last Knight. Disappointing.


--------------------------------------------- Result 2 ---------------------------------------------
[[0 (55%)]] --> [[[SKIPPED]]]

Forrest Gump is a heartwarming journey through history. Tom Hanks delivers an unforgettable performance.





[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:  30%|███       | 3/10 [00:48<01:52, 16.11s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 2 / 3:  30%|███       | 3/10 [00:48<01:52, 16.11s/it]

...

[[0 (59%)]] --> [[1 (51%)]]

[[The]] Emoji Movie is a [[complete]] [[disappointment]]. The [[plot]] is weak, and it [[feels]] like one [[big]] [[advertisement]]. [[A]] [[waste]] of time.

[[Both]] Emoji Movie is a [[fulfills]] [[dissatisfaction]]. The [[plots]] is weak, and it [[reckon]] like one [[largest]] [[commercials]]. [[para]] [[vandalize]] of time.


--------------------------------------------- Result 7 ---------------------------------------------
[[0 (56%)]] --> [[[SKIPPED]]]

Avatar is visually breathtaking, but the story is somewhat predictable. Still, it's a cinematic experience.





[Succeeded / Failed / Skipped / Total] 1 / 2 / 4 / 7:  80%|████████  | 8/10 [01:27<00:21, 10.99s/it]
[Succeeded / Failed / Skipped / Total] 1 / 2 / 5 / 8:  80%|████████  | 8/10 [01:27<00:21, 10.99s/it]
[Succeeded / Failed / Skipped / Total] 1 / 2 / 5 / 8:  90%|█████████ | 9/10 [01:28<00:09,  9.78s/it]
[Succeeded / Failed / Skipped / Total] 1 / 2 / 6 / 9:  90%|█████████ | 9/10 [01:28<00:09,  9.78s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[0 (54%)]] --> [[[SKIPPED]]]

Mind-bending and visually stunning! Inception keeps you guessing from start to finish. Christopher Nolan at his best.


--------------------------------------------- Result 9 ---------------------------------------------
[[0 (55%)]] --> [[[SKIPPED]]]

The Dark Knight is a dark and gripping superhero film. Heath Ledger's Joker is iconic. A must-see.





[Succeeded / Failed / Skipped / Total] 1 / 2 / 6 / 9: 100%|██████████| 10/10 [01:28<00:00,  8.82s/it]
[Succeeded / Failed / Skipped / Total] 1 / 2 / 7 / 10: 100%|██████████| 10/10 [01:28<00:00,  8.82s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[0 (55%)]] --> [[[SKIPPED]]]

This film is a masterpiece! The story is incredibly moving, and the performances are outstanding. It's a true classic.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 30.0%  |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 33.33% |
| Average perturbed word %:     | 40.91% |
| Average num. words per input: | 17.3   |
| Avg num queries:              | 213.33 |
+-------------------------------+--------+


In [20]:
import pandas as pd
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch import nn
import torch
import math
import textattack
import random


#torch.cuda.is_available = lambda : False
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.is_available = lambda : False
textattack.shared.utils.device = "cuda:0"

class ClassificationModel(nn.Module):
    def __init__(self, model, pos_prompt, neg_prompt):
        super(ClassificationModel, self).__init__()
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.model = GPT2LMHeadModel.from_pretrained(model)
        #self.model.to('cuda:0')
        self.model.eval()
        self.pos_prompt = pos_prompt
        self.neg_prompt = neg_prompt

    def score(self, prompt, sentence, model):
        tokenized_prompt = self.tokenizer.encode(prompt , max_length=1024, truncation=True, return_tensors='pt').to('cpu')
        tokenized_all = self.tokenizer.encode(prompt + ' ' + sentence, max_length=1024, truncation=True, return_tensors='pt').to('cpu')

        loss1=model(tokenized_all, labels=tokenized_all).loss 
        loss2 = model(tokenized_prompt, labels=tokenized_prompt).loss*len(tokenized_prompt[0])/len(tokenized_all[0])
        loss = loss1-loss2
        return math.exp(loss)
    

    def forward(self, sentence):
        pos = 0
        neg = 0
        for prompt in self.pos_prompt:
             pos += self.score(prompt, sentence, self.model)#.cpu()
        for prompt in self.neg_prompt:
             neg += self.score(prompt, sentence, self.model)#.cpu()
        #print(neg)
        result = torch.FloatTensor([5000-neg/10.0e+52, 5000-pos/10.0e+52])
        #print('res', result)
        result = torch.softmax(result, 0)
        #print(pos)
        #print(neg)
        #print('result', result)
        if abs(result[0].item()+result[1].item()-1) >= 1e-6:
            print('detected something')
            result = torch.FloatTensor([1,0])
        return torch.softmax(result, 0)


class TorchTokenizer(GPT2Tokenizer):
    def __init__(self):
        super(TorchTokenizer, self).__init__()


class CustomWrapper(textattack.models.wrappers.ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, list_of_texts):
        results = []
        for text in list_of_texts:
          results.append(self.model(text))

        return torch.stack(results)



#model = ClassificationModel('gpt2-xl', ['I loved this movie!','A great film!', "This was an awesome movie!", "This movie was extremely good!", "This was the best movie I have ever seen!", "I found the movie to be very good.", "This film was fantastic!"], ['I hated this movie!', 'A bad film!', "This was a terrible movie!", "This movie was really bad!", "This was the worst movie I have ever seen!", "I found the movie to be very bad.", "This film was boring."]).to('cuda:2')
model = ClassificationModel('gpt2', ['Positive:'], ['Negative:'])
class_model = CustomWrapper(model)


from textattack.datasets import Dataset
from textattack.attack_recipes.textfooler_jin_2019 import TextFoolerJin2019
from textattack import Attacker, AttackArgs

class_model =CustomWrapper(model)
attack = TextFoolerJin2019.build(class_model)

dataset = [
    ["This film is a masterpiece! The story is incredibly moving, and the performances are outstanding. It's a true classic.", 1],
    ["The Godfather is a cinematic gem. The storytelling and performances are top-notch. A true classic in every sense.", 1],
    ["The Emoji Movie is a complete disappointment. The plot is weak, and it feels like one big advertisement. A waste of time.", 0],
    ["Mind-bending and visually stunning! Inception keeps you guessing from start to finish. Christopher Nolan at his best.", 1],
    ["Twilight is a guilty pleasure for some, but the acting and dialogue are cringe-worthy. Not a cinematic masterpiece.", 0],
    ["Forrest Gump is a heartwarming journey through history. Tom Hanks delivers an unforgettable performance.", 1],
    ["Explosions and CGI can't make up for the lackluster story in Transformers: The Last Knight. Disappointing.", 0],
    ["The Dark Knight is a dark and gripping superhero film. Heath Ledger's Joker is iconic. A must-see.", 1],
    ["Avatar is visually breathtaking, but the story is somewhat predictable. Still, it's a cinematic experience.", 1],
    ["The Room is so bad that it's almost good. The unintentional humor makes it a cult classic.", 1]
]
random.shuffle(dataset)

attacker = Attacker(attack, textattack.datasets.Dataset(dataset[:10]), AttackArgs(num_examples=10))
attacker.attack_dataset()

textattack: Unknown if model of class <class '__main__.ClassificationModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       




  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [02:00<18:05, 120.61s/it]


[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  10%|█         | 1/10 [02:00<18:05, 120.62s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (50%)]] --> [[[FAILED]]]

Explosions and CGI can't make up for the lackluster story in Transformers: The Last Knight. Disappointing.







[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  20%|██        | 2/10 [02:01<08:04, 60.51s/it] 


[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:  20%|██        | 2/10 [02:01<08:04, 60.51s/it]

...




[Succeeded / Failed / Skipped / Total] 0 / 3 / 5 / 8:  90%|█████████ | 9/10 [05:33<00:37, 37.10s/it]


[Succeeded / Failed / Skipped / Total] 0 / 3 / 6 / 9:  90%|█████████ | 9/10 [05:33<00:37, 37.10s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[0 (50%)]] --> [[[SKIPPED]]]

The Dark Knight is a dark and gripping superhero film. Heath Ledger's Joker is iconic. A must-see.







[Succeeded / Failed / Skipped / Total] 0 / 3 / 6 / 9: 100%|██████████| 10/10 [05:34<00:00, 33.44s/it]


[Succeeded / Failed / Skipped / Total] 0 / 3 / 7 / 10: 100%|██████████| 10/10 [05:34<00:00, 33.44s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[0 (50%)]] --> [[[SKIPPED]]]

This film is a masterpiece! The story is incredibly moving, and the performances are outstanding. It's a true classic.



+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 3     |
| Number of skipped attacks:    | 7     |
| Original accuracy:            | 30.0% |
| Accuracy under attack:        | 30.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 17.3  |
| Avg num queries:              | 250.0 |
+-------------------------------+-------+



/opt/conda/lib/python3.10/site-packages/textattack/metrics/attack_metrics/words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
